In [12]:
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env into environment

True

In [13]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752855565, 'localtime': '2025-07-18 09:19'}, 'current': {'last_updated_epoch': 1752855300, 'last_updated': '2025-07-18 09:15', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 265, 'wind_dir': 'W', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 13.0, 'windchill_f': 55.5, 'heatindex_c': 13.5, 'heatindex_f': 56.4, 'dewpoint_c': 12.0, 'dew

In [14]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-pro", model_provider="google_genai")

In [15]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [16]:
model_with_tools = model.bind_tools(tools)

In [17]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [18]:
query = "Search for the weather in San Francisco"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in San Francisco'}, 'id': 'cece1e39-a918-45c0-b0d4-a1758ccbef60', 'type': 'tool_call'}]


In [19]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [20]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [21]:
input_message = {"role": "user", "content": "Search for the weather in San Francisco"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in San Francisco
================================== Ai Message ==================================
Tool Calls:
  tavily_search (054bdbba-0ec5-45c1-b1a9-e675c8efc5dc)
 Call ID: 054bdbba-0ec5-45c1-b1a9-e675c8efc5dc
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752856180, 'localtime': '2025-07-18 09:29'}, 'current': {'last_updated_epoch': 1752855300, 'last_updated': '2025-07-18 09:15', 'temp_c': 15.6, 'temp_f': 

In [22]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in San Francisco
================================== Ai Message ==================================
Tool Calls:
  tavily_search (e2c557a5-7d7f-4a54-b36f-584537600095)
 Call ID: e2c557a5-7d7f-4a54-b36f-584537600095
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752856180, 'localtime': '2025-07-18 09:29'}, 'current': {'last_updated_epoch': 1752855300, 'last_updated': '2025-07-18 09:15', 'temp_c': 15.6, 'temp_f': 

In [23]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The current weather in San Francisco is |60.1°F (15.6°C) and partly cloudy. The wind is blowing from the west at| 5.8 mph, and the humidity is at 83%.|

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I help you today?


In [27]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob.


In [28]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not have access to your name.
